# Pantheon Data Collection

In [1]:
import sys,os,glob
from astropy.io import fits
from astropy.table import Table
from astropy.nddata import extract_array
from astropy.coordinates import SkyCoord
from astropy import wcs
from astropy.wcs.utils import skycoord_to_pixel
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.visualization import (simple_norm,LinearStretch)
import pandas as pd
import space_phot



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    


/Users/bhoomika_khatri/Documents/STScI/STScI_pantheon/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Open and read the unmatched pantheon dataset
df = pd.read_csv('unmatched_pantheon(in).csv')

print(df.dtypes)
df_string = df.astype(str)
print(df_string.dtypes)

SNID           object
IAUC           object
host           object
RA            float64
Dec           float64
RA_host       float64
Dec_host      float64
zhel          float64
zcmb          float64
zhelerr       float64
zHD           float64
zHDerr        float64
PV              int64
vpecerr         int64
RA_group      float64
Dec_group     float64
zhel_group    float64
zcmb_group    float64
zHD_group     float64
PV_group      float64
in_group        int64
has_host        int64
is_SNz          int64
dtype: object
SNID          object
IAUC          object
host          object
RA            object
Dec           object
RA_host       object
Dec_host      object
zhel          object
zcmb          object
zhelerr       object
zHD           object
zHDerr        object
PV            object
vpecerr       object
RA_group      object
Dec_group     object
zhel_group    object
zcmb_group    object
zHD_group     object
PV_group      object
in_group      object
has_host      object
is_SNz        obje

In [3]:
#MAST takes RA and Dec as a string, creating the string representation of each supernovae in a new column called "resolved_coord"
df = df.assign(resolved_coord = df_string['RA'] + " " + df_string['Dec'])
print(df['resolved_coord'])

0         167.62425 55.16083
1         248.61546 76.02967
2          358.38658 8.11739
3          21.34542 34.02514
4         144.48792 25.49481
                ...         
1856     53.259174 -26.85409
1857     52.695011 -28.64933
1858    54.343502 -28.826429
1859     55.340694 -29.14934
1860    52.889896 -28.912092
Name: resolved_coord, Length: 1861, dtype: object


In [30]:
#Testing the observation table prior to creating a query
i = 12
print(df['resolved_coord'][i]) 

obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][i],
                                        radius="0.006 deg",
                                        intentType = 'science',
                                        filters = ['F1*'],
                                        #calibration = 1,
                                        obs_collection=['HST', 'JWST'])
obs_table = obs_table[obs_table['calib_level']==3]
print(len(obs_table))

211.65662 -5.43942
19


In [4]:
#Creating a function that will create a folder for the data (named the resolved coordinate) and keep the query there
valid_points = []
def save_images_supernova(resolved_coord_index):
    """
    If there are images for the supernovae in JWST or HST at the resolved query, create a folder and keep the data there
    Parameters:
        resolved_coord_index: integer, the index number of the query
    Returns:
        Folder of Data at requested file path (pantheon_data_folder/{resolved coord})
    """
    print(resolved_coord_index, df['resolved_coord'][resolved_coord_index])

    #try the the query, if a error is thrown then print "No Data Points" and "skip" this data point, else make folder and store data
    try:
        obs_table = Observations.query_criteria(coordinates=df['resolved_coord'][resolved_coord_index],
                                            radius="0.006 deg",
                                            intentType = 'science',
                                            filters = ['F1*'],
                                            obs_collection=['HST', 'JWST'])
        obs_table = obs_table[obs_table['calib_level']==3]
        data_products = Observations.get_product_list(obs_table)
    except: 
        print("No Data Points :(")
        exit
    else:
        print("Has Data Points!")
        valid_points.append(resolved_coord_index)
        """
        folder_path = "pantheon_data_folder/{}".format(df['resolved_coord'][resolved_coord_index])
        os.makedirs(folder_path, exist_ok=True)
        #Fits is what we need for space_phot, jpg to see image
        manifest = Observations.download_products(data_products, download_dir=folder_path, extension=['fits'])
        print(manifest)
        """

In [18]:
save_images_supernova(12)

12 211.65662 -5.43942
Has Data Points!


In [ ]:
#len(df['resolved_coord'])
#12,13,14,15,17
for i in range(len(df['resolved_coord'])):
    save_images_supernova(i)

0 167.62425 55.16083
No Data Points :(
1 248.61546 76.02967
No Data Points :(
2 358.38658 8.11739
No Data Points :(
3 21.34542 34.02514
No Data Points :(
4 144.48792 25.49481
No Data Points :(
5 149.754 17.82011
No Data Points :(
6 137.38825 50.28092
No Data Points :(
7 347.68058 7.56956
No Data Points :(
8 179.35388 25.2025
No Data Points :(
9 153.92629 55.66853
No Data Points :(
10 216.16883 26.62631
No Data Points :(
11 236.75312 -0.99042
No Data Points :(
12 211.65662 -5.43942
Has Data Points!
13 244.12658 35.70839
Has Data Points!
14 198.25142 -19.519
Has Data Points!
15 352.12667 22.42889
Has Data Points!
16 27.43167 32.62861
No Data Points :(
17 50.52379 -15.40089
Has Data Points!
18 196.98025 34.08514
No Data Points :(
19 311.82742 0.31267
No Data Points :(
20 124.99512 62.82033
No Data Points :(
21 99.31379 49.85283
No Data Points :(
22 142.09408 27.44464
No Data Points :(
23 109.49133 9.69303
No Data Points :(
24 179.05904 60.52197
No Data Points :(
25 218.64917 59.33439
Has 

In [ ]:
df_vp = pd.DataFrame(valid_points)
df_vp.to_csv('valid_points_index_list.csv', index=False)

In [ ]:
def HST_aperture(resolved_coordinate_index):
    """
    For HST images, use the aperature techinique to go through and find PSF photometry
    Parameters:
        resolved_coordinate_index: integer, the index number of the query
    Returns:
        - Full image of the field
        - Image of the supernova
        - Print final aperture result
    """
    RA = df['RA'][resolved_coordinate_index]
    DEC = df['Dec'][resolved_coordinate_index]
    resolved_coord = df['resolved_coord'][resolved_coordinate_index]

    files = glob.glob("pantheon_data_folder/{}/mastDownload/HST/*/*flt.fits".format(resolved_coord))
    #If there are NO HST IMAGES exit
    if len(files) == 0:
        print("No HST images")
        exit

    #RIGHT NOW THIS IS 0 THIS WILL NEED TO CHANGE
    image_num = 0

    #Takes the data from the file

    ref_image = files[image_num]
    ref_fits = fits.open(ref_image)
    ref_data = fits.open(ref_image)['SCI',1].data

    #Creating the visual of the image region
    norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)
    plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.title("Image of Full File {} for coord {}".format(image_num, resolved_coord))
    plt.show()


    #Taking the supernova position and zooming in on this image + creating image
    sn_location = SkyCoord(ra=RA*u.degree, dec=DEC*u.degree, frame='icrs')
    ref_y,ref_x = skycoord_to_pixel(sn_location,wcs.WCS(ref_fits['SCI',1],ref_fits))

    #400 by 400 image
    full_cutout = extract_array(ref_data,(400,400),(ref_x,ref_y))
    ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
    norm1 = simple_norm(full_cutout,stretch='linear',vmin=-1,vmax=10)
    #Image in black and grey
    plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
    #Target Region over it in Red
    plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='reds')
    plt.title("400 by 400 {}".format(resolved_coord))
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.show()

    #more Zoomed in
    ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
    norm1 = simple_norm(ref_cutout,stretch='linear',vmin=-1,vmax=10)
    plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
    plt.title(resolved_coord)
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.show()
    print (files)

    #Final Aperature result
    hst_obs = space_phot.observation2(files)
    hst_obs.aperture_photometry(sn_location,radius=3,
                        skyan_in=5,skyan_out=7)
    print(hst_obs.aperture_result.phot_cal_table)


In [ ]:
#Create a function that will take the index of the resolved coordinate index and do analysis on the files in that subgroup

def HST_PSF_function(resolved_coordinate_index):
    """
    For Hubble Space Telescope (HST) images, go through and find fluxes of the supernova after cleaning (PSF)
    Params:
        resolved_coordinate_index: integer, the index number of the query
    Returns:
        - Full image of the field
        - Image of the supernova
        - PSF model
        - PSF photometry overall
    """
    
    RA = df['RA'][resolved_coordinate_index]
    DEC = df['Dec'][resolved_coordinate_index]
    resolved_coord = df['resolved_coord'][resolved_coordinate_index]

    files = glob.glob("pantheon_data_folder/{}/mastDownload/HST/*/*flt.fits".format(resolved_coord))
    #If there are NO HST IMAGES exit
    if len(files) == 0:
        print("No HST images")
        exit
    
    #RIGHT NOW THIS IS 0 THIS WILL NEED TO CHANGE
    image_num = 0

    #Takes the data from the file

    ref_image = files[image_num]
    ref_fits = fits.open(ref_image)
    ref_data = fits.open(ref_image)['SCI',1].data

    #Creating the visual of the image region
    norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)
    plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.title("Image of File {} for coord {}".format(image_num, resolved_coord))
    plt.show()

    #Taking the supernova position and zooming in on this image + creating image
    sn_location = SkyCoord(ra=RA*u.degree, dec=DEC*u.degree, frame='icrs')
    ref_y,ref_x = skycoord_to_pixel(sn_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
    ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
    norm1 = simple_norm(ref_cutout,stretch='linear',vmin=-1,vmax=10)
    plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
    plt.title(resolved_coord)
    plt.gca().tick_params(labelcolor='none',axis='both',color='none')
    plt.show()
    print (files)
    """
    hst_obs = space_phot.observation2(files)
    print(hst_obs)
    psfs = space_phot.get_hst_psf(hst_obs,sn_location)
    plt.imshow(psfs[0].data)
    plt.show()
    """


    

HST_PSF_function(2)


In [ ]:
files = glob.glob('pantheon_data_folder/*/mastDownload/HST/*/*flt.fits')
print(len(files))

ref_image = files[0]
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
sn_location = SkyCoord('21:29:40.2110','+0:05:24.154',unit=(u.hourangle,u.deg))
ref_y,ref_x = skycoord_to_pixel(sn_location,wcs.WCS(ref_fits['SCI',1],ref_fits))
ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
norm1 = simple_norm(ref_cutout,stretch='linear',min_cut=-1,max_cut=10)
plt.imshow(ref_cutout, origin='lower',
                      norm=norm1,cmap='gray')
plt.title('SN2022riv')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()

In [ ]:
resolved_coordinate_index = 2
RA = df['RA'][resolved_coordinate_index]
DEC = df['Dec'][resolved_coordinate_index]
resolved_coord = df['resolved_coord'][resolved_coordinate_index]

files = glob.glob("pantheon_data_folder/{}/mastDownload/HST/*/*flt.fits".format(resolved_coord))

print(len(files))
#If there are NO HST IMAGES exit


#RIGHT NOW THIS IS 0 THIS WILL NEED TO CHANGE
ref_image = files[0]
ref_fits = fits.open(ref_image)
ref_data = fits.open(ref_image)['SCI',1].data
norm1 = simple_norm(ref_data,stretch='linear',min_cut=-1,max_cut=10)

plt.imshow(ref_data, origin='lower',norm=norm1,cmap='gray')
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()
"""
sn_location = SkyCoord(ra=RA*u.degree, dec=DEC*u.degree, frame='icrs')
ref_y,ref_x = skycoord_to_pixel(sn_location,wcs.WCS(ref_fits['SCI',1],ref_fits)) # pyright: ignore[reportUndefinedVariable]
ref_cutout = extract_array(ref_data,(11,11),(ref_x,ref_y))
norm1 = simple_norm(ref_cutout,stretch='linear',vmin=-1,vmax=10)
plt.imshow(ref_cutout, origin='lower',
                    norm=norm1,cmap='gray')
plt.title(resolved_coord)
plt.gca().tick_params(labelcolor='none',axis='both',color='none')
plt.show()
"""